# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 8:30AM,247101,10,SOTAH0000786,"Nextsource Biotechnology, LLC",Released
1,Sep 22 2022 3:41PM,247102,12,HH8039,Hush Hush,Released
2,Sep 22 2022 3:26PM,247096,10,0086002602,ISDIN Corporation,Released
3,Sep 22 2022 3:26PM,247096,10,0086002603,ISDIN Corporation,Released
4,Sep 22 2022 3:26PM,247096,10,0086002604,ISDIN Corporation,Released
5,Sep 22 2022 3:26PM,247096,10,0086002632,ISDIN Corporation,Released
6,Sep 22 2022 3:26PM,247096,10,0086002629,ISDIN Corporation,Released
7,Sep 22 2022 3:26PM,247096,10,0086002631,ISDIN Corporation,Released
8,Sep 22 2022 3:26PM,247096,10,0086002633,ISDIN Corporation,Released
9,Sep 22 2022 3:26PM,247096,10,0086002634,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,247096,Released,44
28,247097,Executing,1
29,247100,Released,1
30,247101,Released,1
31,247102,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247096,NaN,NaN,44.0
247097,NaN,1.0,NaN
247100,NaN,NaN,1.0
247101,NaN,NaN,1.0
247102,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246997,0.0,1.0,0.0
246998,0.0,1.0,0.0
246999,0.0,0.0,1.0
247005,6.0,3.0,2.0
247015,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246997,0,1,0
246998,0,1,0
246999,0,0,1
247005,6,3,2
247015,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246997,0,1,0
1,246998,0,1,0
2,246999,0,0,1
3,247005,6,3,2
4,247015,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246997,,1,
1,246998,,1,
2,246999,,,1
3,247005,6,3,2
4,247015,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC"
1,Sep 22 2022 3:41PM,247102,12,Hush Hush
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation
46,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company
47,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc."
48,Sep 22 2022 2:55PM,246997,21,"NBTY Global, Inc."
49,Sep 22 2022 2:53PM,246998,21,"NBTY Global, Inc."
50,Sep 22 2022 2:48PM,247097,19,"GCH Granules USA, Inc."
51,Sep 22 2022 2:47PM,247095,19,"SD Head USA, LLC"
52,Sep 22 2022 2:40PM,247092,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC",,,1
1,Sep 22 2022 3:41PM,247102,12,Hush Hush,,,1
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation,,,44
3,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company,,,1
4,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc.",,,1
5,Sep 22 2022 2:55PM,246997,21,"NBTY Global, Inc.",,1,
6,Sep 22 2022 2:53PM,246998,21,"NBTY Global, Inc.",,1,
7,Sep 22 2022 2:48PM,247097,19,"GCH Granules USA, Inc.",,1,
8,Sep 22 2022 2:47PM,247095,19,"SD Head USA, LLC",,1,
9,Sep 22 2022 2:40PM,247092,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 22 2022 3:41PM,247102,12,Hush Hush,1,,
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation,44,,
3,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company,1,,
4,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc.",1,,
5,Sep 22 2022 2:55PM,246997,21,"NBTY Global, Inc.",,1,
6,Sep 22 2022 2:53PM,246998,21,"NBTY Global, Inc.",,1,
7,Sep 22 2022 2:48PM,247097,19,"GCH Granules USA, Inc.",,1,
8,Sep 22 2022 2:47PM,247095,19,"SD Head USA, LLC",,1,
9,Sep 22 2022 2:40PM,247092,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 22 2022 3:41PM,247102,12,Hush Hush,1,,
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation,44,,
3,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company,1,,
4,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Sep 22 2022 3:41PM,247102,12,Hush Hush,1.0,NaN,NaN
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation,44.0,NaN,NaN
3,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company,1.0,NaN,NaN
4,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247101,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Sep 22 2022 3:41PM,247102,12,Hush Hush,1.0,0.0,0.0
2,Sep 22 2022 3:26PM,247096,10,ISDIN Corporation,44.0,0.0,0.0
3,Sep 22 2022 3:21PM,247100,19,Eli Lilly and Company,1.0,0.0,0.0
4,Sep 22 2022 2:59PM,246999,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1729395,57.0,12.0,7.0
12,247102,1.0,0.0,0.0
15,494060,2.0,4.0,36.0
19,1235463,3.0,2.0,0.0
20,494079,35.0,1.0,0.0
21,740994,1.0,2.0,0.0
22,988336,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1729395,57.0,12.0,7.0
1,12,247102,1.0,0.0,0.0
2,15,494060,2.0,4.0,36.0
3,19,1235463,3.0,2.0,0.0
4,20,494079,35.0,1.0,0.0
5,21,740994,1.0,2.0,0.0
6,22,988336,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,12.0,7.0
1,12,1.0,0.0,0.0
2,15,2.0,4.0,36.0
3,19,3.0,2.0,0.0
4,20,35.0,1.0,0.0
5,21,1.0,2.0,0.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,12,Released,1.0
2,15,Released,2.0
3,19,Released,3.0
4,20,Released,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,7.0,0.0,36.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,4.0,2.0,1.0,2.0,0.0
Released,57.0,1.0,2.0,3.0,35.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,7.0,0.0,36.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,4.0,2.0,1.0,2.0,0.0
2,Released,57.0,1.0,2.0,3.0,35.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,7.0,0.0,36.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,4.0,2.0,1.0,2.0,0.0
2,Released,57.0,1.0,2.0,3.0,35.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()